### Seleção de Modelos

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

- Nesta etapa vamos analisar a performance de diferentes tipos de modelos como **Árvores de Decisões**, **K Vizinhos**, **Regressões Logísticas** e **Método de Naive Bayes** presentes no site [Scikit-Learn](https://scikit-learn.org/)

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
from sklearn.preprocessing import StandardScaler

df_train = pd.read_csv("dados/treino_tratado.csv", header=0)
df_test = pd.read_csv("dados/teste_tratado.csv", header=0)

data = df_train.to_numpy()
n_row, n_col = data.shape
X = data[:,1:n_col]
y = data[:,0]
X_test = df_test.to_numpy()

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)


- Primeiro testamos o modelos de [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), utilizando o método de **Random Search**.

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

params = { "bootstrap" : [True],
          "max_depth": [ 6, 8, 10],
          "n_estimators": [100, 200, 400, 600, 800, 1000],
          "min_samples_leaf" : [2,3,4],
          "min_samples_split": [2,3,4,5],
          "max_features": ["auto", "sqrt", "log2"]
}

rfc_RS = RandomizedSearchCV(rfc, params, cv=5, n_iter=32, scoring="accuracy", random_state=42, n_jobs=-1, )

model = rfc_RS.fit(X, y)

print(f"O melhor resultado do Random Forest foi {model.best_score_} \nUtilizando os seguintes parâmetros: {model.best_estimator_} ")


/home/edmurcn/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/edmurcn/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/edmurcn/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it 

O melhor resultado do Random Forest foi 0.8283095850856819 
Utilizando os seguintes parâmetros: RandomForestClassifier(max_depth=8, max_features='auto', min_samples_leaf=2,
                       n_estimators=800) 


- Agora, testamos o modelo de classificação por [K vizinhos](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors).

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors()

